In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import re
from wordcloud import WordCloud, STOPWORDS
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk

In [2]:
StocksDF = pd.read_csv('TESLASTOCKS.csv')
StocksDF['Date'] = pd.to_datetime(StocksDF['Date'], dayfirst=True)

In [3]:
Percent_Change = [StocksDF["Percent Change"][i][:-1] for i in range(len(StocksDF["Percent Change"]))]
StocksDF = StocksDF.drop(columns = "Percent Change")
StocksDF["Percent Change"] = Percent_Change

In [4]:
TweetsDF = pd.read_csv('TWEETS.csv')
TweetsDF = TweetsDF.rename(columns = {'Timestamp':'Date'})
TweetsDF = TweetsDF.drop(['Emojis','Retweets','UserScreenName', 'UserName','Comments','Likes','Image link'],axis=1)
TweetsDF['Date'] = pd.to_datetime(TweetsDF['Date'], dayfirst=True)

In [5]:
df = pd.merge(StocksDF, TweetsDF, on = "Date")
df = df.drop_duplicates(subset='Date', keep = "first")
df.reset_index(drop=True, inplace=True)

In [6]:
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Price Change,Percent Change,Tweet,text,Tweet URL
0,2018-01-03,64.199997,65.050003,63.110001,63.450001,63.450001,22607500,0.628006,1,2992,Come work at the biggest & most advanced facto...,https://twitter.com/elonmusk/status/9484282744...
1,2018-01-05,63.324001,63.448002,62.400002,63.316002,63.316002,22956000,-0.261997,0,2994,"Falcon Heavy goes vertical\n0:19\n2M views\n2,...",https://twitter.com/elonmusk/status/9490754750...
2,2018-01-08,63.200001,67.403999,63.099998,67.281998,67.281998,49297000,3.955997,6,2996,"Long exposure of rocket ascent, reentry from s...",https://twitter.com/elonmusk/status/9504162449...
3,2018-01-09,67.031998,67.760002,65.480003,66.737999,66.737999,35733000,0.356003,1,2998,Replying to \n@PPathole\n @Tesla\n and 2 other...,https://twitter.com/elonmusk/status/9506003636...
4,2018-01-12,67.725998,68.082001,66.734001,67.244003,67.244003,24125500,-0.879997,-1,3006,Replying to \n@ID_AA_Carmack\n and \n@Danielle...,https://twitter.com/elonmusk/status/9518938946...


In [7]:
def cleanTxt(text):
    text = re.sub('@[A-Za-z0–9]+', '', text) #Removing @mentions
    text = re.sub('#', '', text) # Removing '#' hash tag
    text = re.sub('RT[\s]+', '', text) # Removing RT
    text = re.sub('https?:\/\/\S+', '', text) # Removing hyperlink
df["text"] = df["text"].apply(cleanTxt)

<>:4: SyntaxWarning: invalid escape sequence '\s'
<>:5: SyntaxWarning: invalid escape sequence '\/'
<>:4: SyntaxWarning: invalid escape sequence '\s'
<>:5: SyntaxWarning: invalid escape sequence '\/'
/var/folders/ff/g92mc5sn1qj8knmdc1kc6x2r0000gn/T/ipykernel_3181/2909126462.py:4: SyntaxWarning: invalid escape sequence '\s'
  text = re.sub('RT[\s]+', '', text) # Removing RT
/var/folders/ff/g92mc5sn1qj8knmdc1kc6x2r0000gn/T/ipykernel_3181/2909126462.py:5: SyntaxWarning: invalid escape sequence '\/'
  text = re.sub('https?:\/\/\S+', '', text) # Removing hyperlink


In [8]:
def percentage(part,whole):
    return 100 * float(part)/float(whole)

#Assigning Initial Values
positive = 0
negative = 0
neutral = 0
#Creating empty lists
tweet_list1 = []
neutral_list = []
negative_list = []
positive_list = []
pos_increased_value = []
neg_increased_value = []
pos_decreased_value = []
neg_decreased_value = []
pos_max_value = []
neg_max_value = []
pos_min_value = []
neg_min_value = []

#Iterating over the tweets in the dataframe
i = 0
for tweet in df['text']:
    tweet_list1.append(tweet)
    analyzer = SentimentIntensityAnalyzer().polarity_scores(tweet)
    neg = analyzer['neg']
    neu = analyzer['neu']
    pos = analyzer['pos']
    comp = analyzer['compound']

    if neg > pos:
        negative_list.append(tweet) #appending the tweet that satisfies this condition
        negative += 1 #increasing the count by 1
        neg_max_value.append(df['High'][i])
        neg_min_value.append(df['Low'][i])
        if df['Percent Change'][i] > 0:
            neg_increased_value.append(df['Percent Change'][i])
        elif df['Percent Change'][i] <= 0:
            neg_decreased_value.append(df['Percent Change'][i])
    if pos > neg:
        positive_list.append(tweet) #appending the tweet that satisfies this condition
        positive += 1 #increasing the count by 1
        pos_max_value.append(df['High'][i])
        pos_min_value.append(df['Low'][i])
        if df['Percent Change'][i] > 0:
            pos_increased_value.append(df['Percent Change'][i])
        elif df['Percent Change'][i] <= 0:
            pos_decreased_value.append(df['Percent Change'][i])
    elif pos == neg:
        neutral_list.append(tweet) #appending the tweet that satisfies this condition
        neutral += 1 #increasing the count by 1 
    
    i += 1

positive = percentage(positive, len(df)) #percentage is the function defined above
negative = percentage(negative, len(df))
neutral = percentage(neutral, len(df))

LookupError: 
**********************************************************************
  Resource [93mvader_lexicon[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('vader_lexicon')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93msentiment/vader_lexicon.zip/vader_lexicon/vader_lexicon.txt[0m

  Searched in:
    - '/Users/Ansh/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.12/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.12/share/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.12/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************
